In [0]:
%tensorflow_version 1.x
import numpy as np
from keras.models import Sequential
import keras
import tensorflow as tf
from keras.layers import Dense, Convolution2D, SpatialDropout2D, MaxPooling2D,Input, Flatten, Activation, Dropout
from keras.layers.core import Lambda
from keras import backend as K
from keras.regularizers import l2
from keras.models import Model
from scipy.stats import entropy
import os

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [0]:
def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        Mean of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 2))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    prediction = result.mean(axis=0)
    return prediction

def predict_pool_with_uncertainty(f, x, n_iter=50):
    """Function generating and returning non-deterministic predictions using MC dropout
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        All MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 2))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    return result

def fixed_mask_forward_pass (model, forward_pass_input, num_masks, dropout_prob, conv_masks, dense_masks):
      """Makes model predictions with J dropout masks that are fixed across points to enable estimation of Var(p_{sample})
    Function is specific to the given Keras model.
    #Arguments
        model: keras model
        forward_pass_input: X_{sample}
        num_masks: J, the number of dropout masks being used in estimation of Var(p_{sample}) and calculation of the EI acquisition function
        dropout_prob: dropout probability 
        conv_masks: first set of dropout masks, applied after MaxPooling2D layer
        dense_masks: second set of dropout masks, applied after the first Dense layer
    #Returns
        MC dropout predictions across sample points enabling estimation of Var(p_{sample}), i.e. \hat{p}_{sample}
    """
    conv = K.function([model.layers[0].input, K.learning_phase()],
                    [model.layers[4].output])

    dense_1 = K.function([model.layers[6].input, K.learning_phase()],
                   [model.layers[8].output])

    dense_2 = K.function([model.layers[10].input, K.learning_phase()],
                   [model.layers[11].output])
    num_points = len(forward_pass_input)
    forward_pass_input = np.expand_dims(forward_pass_input, axis=1)
    conv_output = np.array(conv((forward_pass_input, 1)))
    dense_1_input = apply_dropout_masks(conv_output, conv_masks)
    if num_points == 1:
        dense_1_input = np.expand_dims(dense_1_input, axis=1)
    dense_1_output = multi_mask_predict(dense_1, dense_1_input)
    dense_2_input = apply_dropout_masks(dense_1_output, dense_masks)
    if num_points == 1:
        dense_2_input = np.expand_dims(dense_2_input, axis=1)
    dense_2_output = multi_mask_predict(dense_2, dense_2_input)
    return np.squeeze(dense_2_output)

def initialize_model (train_data_indices):
  """Used to initialize a Keras model in each active learning iteration; enables model retraining from scratch in each iteration
  #Arguments:
    train_data_indices: used to determine the current training set size, which affects the weight decay
  #Returns:
    A new initialized Keras model
  """
  X_train = np.expand_dims(X_train_All[train_data_indices], axis=1)
  y_train = y_train_All[train_data_indices]
  y_train[y_train==7] = 1
  y_train[y_train==9] = 0
  y_train = keras.utils.to_categorical(y_train, num_classes=2)
  train_size = y_train.shape[0]
  Weight_Decay = 0.0005/train_size
  dropout_prob = 0.30
  batch_size=128
  nb_filters = 35
  nb_pool = 3
  nb_conv = 4
  img_rows = img_cols = 28
  nb_classes = 2
  model = Sequential()
  model.add(Convolution2D(nb_filters, nb_conv, strides=1, data_format="channels_first", input_shape=(1, img_rows, img_cols)))
  model.add(Activation('relu'))
  model.add(Convolution2D(nb_filters, nb_conv, data_format="channels_first", strides=2))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool), data_format="channels_first"))
  model.add(Dropout(dropout_prob))
  model.add(Flatten())
  model.add(Dense(128, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('relu'))
  model.add(Dropout(dropout_prob))
  model.add(Dense(nb_classes, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  return model

def run_model (train_data_indices):
  """Trains and Keras model with the training points specified by train_data_indices and evaluates model on test data
  #Arguments:
      train_data_indices: indices of current training points within X_train_All
  #Returns:
      Test accuracy and Keras model
  """
  X_train = np.expand_dims(X_train_All[train_data_indices], axis=1)
  y_train = y_train_All[train_data_indices]
  y_train[y_train==7] = 1
  y_train[y_train==9]=0
  y_train = keras.utils.to_categorical(y_train, num_classes=2)
  model = initialize_model(train_data_indices)
  model.fit(X_train, y_train, epochs=300, batch_size=128, verbose=0)
  f = K.function([model.layers[0].input, K.learning_phase()],[model.layers[-1].output])
  y_test_output = predict_with_uncertainty(f, X_test, n_iter=100)
  y_test_predictions = np.argmax(y_test_output, axis=1)
  return [np.sum(y_test_predictions==y_test_original)/(y_test_original.shape[0]), model]



def create_dropout_masks(dim, drop_prob):
    """Creates dropout masks that are fixed across all model inputs for use in estimation of Var(Y_{sample})
    Note: this function is specific to keras models used in this experiment; number of dropout masks J is deduced from dim.
    #Arguments
        dim: the numpy array shape of values entering the generated dropout masks for all points across all J fixed dropout masks
        drop_prob: the dropout probability
    #Returns
        dropout masks used in estimation of Var(p_{sample})
    """
    if (len(dim)==4):
        return 1/(1-drop_prob)*np.random.choice(2, size=((dim[0], 1, 1, dim[3])), p=[drop_prob, 1-drop_prob])
    return 1/(1-drop_prob)*np.random.choice(2, size=((dim[0], 1, 1, dim[3], dim[4])), p=[drop_prob, 1-drop_prob])

def apply_dropout_masks(a,b):
    return np.squeeze(np.multiply(a,b))

def multi_mask_predict(layer_fn, multi_mask_input):
    """Applies Keras model layers to multiple arrays of layer inputs for all points across J fixed dropout masks
    #Arguments
        layer_fn: keras backend function applying the mapping corresponding to keras model layers
        multi_mask_input: numpy array of layer inputs for all points across J fixed dropout masks
    #Returns
        predictions from layer_fn applied to multi_mask_input as a numpy array
    """
    layer_output = []
    for mask_num in range(multi_mask_input.shape[0]):
        layer_output.append(layer_fn((multi_mask_input[mask_num], 1)))
    return np.array(layer_output)


def batch_bald (model, num_classes, X_cand, num_masks, batchbald_mc_samples, batch_size, dropout_prob):
    """Runs BatchBald, greedily assembling a queried batch of points. This function is Keras model-specific.
    #Arguments
      model: Keras model
      num_classes: number of total classes
      X_cand: candidate points for acquisition from the pool
      num_masks: J, the number of fixed dropout masks used in making predictions on X_cand
      batchbald_mc_samples: number of MC samples from joint distribution over model outputs of queried points
      batch_size: number of queried points per batch
      dropout_prob: dropout probability
    #Returns
      Indices of queried points based on their ordering in X_cand
    """
    forward_pass_input = X_cand
    #Generating fixed mask pool predictions
    conv_masks = 1/(1-dropout_prob)*np.random.choice(2, size=((num_masks, 1, 35, 3, 3)), p=[dropout_prob, 1-dropout_prob])
    dense_masks = 1/(1-dropout_prob)*np.random.choice(2, size=((num_masks, 1, 1, 128)), p=[dropout_prob, 1-dropout_prob])
    k_mc_pool_pred = fixed_mask_forward_pass(model, forward_pass_input, num_masks, dropout_prob, conv_masks, dense_masks).T
    #shape of k_mc_pool_pred is [num_classes, num_candidates, num_masks]
    #Calculating quantities that will be involved in batchbald computations throughout batch assembly
    all_pt_entropies = entropy(k_mc_pool_pred, base=2, axis=0)
    avg_pt_entropies = np.mean(all_pt_entropies, axis=1)
    avg_mc_pool_pred = np.mean(k_mc_pool_pred, axis=2)
    avg_prob_entropies = entropy(avg_mc_pool_pred, base=2, axis=0)
    acq_ind = []
    for itr in range(batch_size):
      #In the first iteration simply compute the BALD objective; no additional MC sampling
      if itr == 0:
        acq_values = avg_prob_entropies - avg_pt_entropies
        acq_ind.append(np.argmax(acq_values))
      else:
        all_acq_values = np.zeros(len(X_cand))
        #y_n-1 sampled from sample over dropout masks ~ probability of y_n-1 | w (i.e. from p(w,y_n-1))
        #Draw batchbald_mc_samples of y hat 1:n-1 
        all_sampled_y = np.zeros((batchbald_mc_samples, itr))
        for sample_itr in range(batchbald_mc_samples):
          mask_ind = np.random.choice(num_masks)
          sampled_y = np.zeros(itr)
          for batch_ind in range(itr):
            prob_vector = k_mc_pool_pred[:, acq_ind[batch_ind], mask_ind]
            sampled_y[batch_ind] = np.random.choice(num_classes, p=prob_vector)
          all_sampled_y[sample_itr,:] = sampled_y
        #Calculating probability of all batchbald_mc_samples y_n-1 sequences under each of the k masks
        p_hat_n_minus_1 = np.zeros((batchbald_mc_samples, num_masks))
        for batchbald_sample_ind in range(batchbald_mc_samples):
          y_n_minus_1 = all_sampled_y[batchbald_sample_ind,:]
          for mask_ind in range(num_masks):
            running_prob = 1.0
            for indiv_acq_ind in range(itr):
              running_prob *= k_mc_pool_pred[int(y_n_minus_1[indiv_acq_ind]), acq_ind[indiv_acq_ind], mask_ind]
            p_hat_n_minus_1[batchbald_sample_ind, mask_ind] = running_prob
        first_term_p_n_minus_1_denom = np.matmul(p_hat_n_minus_1, np.ones((num_masks, 1)))
        #Calculating probability of each new point under each of the k masks (while iterating through candidate points)
        all_joint_entropies = []
        for cand_acq_ind in range(len(X_cand)):
          p_hat_n = k_mc_pool_pred[:, cand_acq_ind,:]
          first_term_numerator = np.matmul(p_hat_n_minus_1, p_hat_n.T)
          first_term = np.divide(first_term_numerator, first_term_p_n_minus_1_denom)
          second_term = np.log2(first_term_numerator/num_masks)
          second_term[np.argwhere(np.isfinite(second_term)==False)] = 0.0
          total_joint_entropy = -1.0 * np.sum(np.multiply(first_term, second_term))/batchbald_mc_samples
          all_joint_entropies.append(total_joint_entropy)
          if cand_acq_ind not in acq_ind:
            curr_batch_ind = acq_ind + [cand_acq_ind]
          else:
            curr_batch_ind = acq_ind
          all_acq_values[cand_acq_ind] = total_joint_entropy - np.sum(avg_pt_entropies[np.array(curr_batch_ind)])
        top_acq_ind = np.flip(np.argsort(all_acq_values))
        found_new_pt = False
        curr_ind = -1
        while found_new_pt == False:
          curr_ind += 1
          if top_acq_ind[curr_ind] not in acq_ind:
            acq_ind.append(top_acq_ind[curr_ind])
            found_new_pt = True
        print('bald curr bs ' + str(itr+1) + ', current acq value: ' + str(all_acq_values[top_acq_ind[curr_ind]]) + ', max joint entropy: ' + str(max(all_joint_entropies)))
    print('function output: ' + str(acq_ind))
    return acq_ind

In [0]:
#Active learning parameters/settings
dropout_prob = 0.3
num_experiments = 3
acq_fn_dropout_iterations = 50
num_acquisitions = 1000
batchbald_mc_samples = 10000
num_candidates = 2500
num_masks = 50
num_classes = 2
batch_size = 100


In [0]:
#Loading data
data_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/"
starting_ind_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/Binary_7_9_AL_Scripts/"
results_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/Binary_7_9_AL_Results/"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
			delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
			delimiter=",") 
y_train_All = train_data[:,0]
y_test = test_data[:,0]
X_train_All = train_data[:,1:].reshape((60000,28,28))
X_test = test_data[:,1:].reshape((10000,28,28)) 
train_ind = np.concatenate((np.argwhere(y_train_All==7), np.argwhere(y_train_All==9))).flatten()
test_ind = np.concatenate((np.argwhere(y_test==7), np.argwhere(y_test==9))).flatten()
y_test = y_test[test_ind]
y_test[y_test==9] = 0
y_test[y_test==7]= 1
y_test_original = y_test
y_test = keras.utils.to_categorical(y_test, num_classes=2)
X_test = np.expand_dims(X_test[test_ind], axis=1)

#Iterating across experiments, each of which begins with a different training set (that is balanced across classes)
for e in range(0,1):
  acc_file = "BatchBald_BS"+str(batch_size)+"_Acc_Ind" + str(e+1) + ".npy"
  ind_file = "BatchBald_BS"+str(batch_size)+"_Ind_Ind" + str(e+1) + ".npy"
  exp_acc = []
  #exp_acc = list(np.load(results_path+acc_file))
  train_data_indices = list(np.load(starting_ind_path + 'trainindices' + str(e+1) + '.npy'))
  #train_data_indices = list(np.load(results_path+ind_file))
  pool_indices = [i for i in train_ind if i not in train_data_indices]
  [init_acc, model] = run_model(train_data_indices)
  exp_acc.append(init_acc)
  num_acquisitions = num_acquisitions - batch_size * (len(exp_acc) - 1)

  #Looping over active learning iterations
  for acq in range(num_acquisitions//batch_size):
    cand_indices = np.random.choice(pool_indices, num_candidates, replace=False)
    X_pool_fn = X_train_All[cand_indices]
    #BatchBald acquisition
    acq_ind_ind = batch_bald(model, num_classes, X_pool_fn, acq_fn_dropout_iterations, batchbald_mc_samples, batch_size, dropout_prob)
    all_acq_ind = cand_indices[acq_ind_ind]
    for acq_ind in all_acq_ind:
      train_data_indices.append(acq_ind)
      pool_indices.remove(acq_ind)
    #Retraining model and saving test accuracy
    [acc, model] = run_model(train_data_indices)
    exp_acc.append(acc)
    print('Experiment ' + str(e+1) +", Total training points: " + str(len(train_data_indices)) + ", exp acc: " + str(exp_acc))
    np.save(results_path+acc_file, np.array(exp_acc))
    np.save(results_path+ind_file, np.array(train_data_indices))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


bald curr bs 2, current acq value: 1.8278066502799808, max joint entropy: 1.9464184145050698
bald curr bs 3, current acq value: 2.582667720373912, max joint entropy: 2.752722934732702
bald curr bs 4, current acq value: 3.290324972413785, max joint entropy: 3.5425430102574893
bald curr bs 5, current acq value: 3.7985157984109597, max joint entropy: 4.164496217362894
bald curr bs 6, current acq value: 4.23305920379928, max joint entropy: 4.662457068388957
bald curr bs 7, current acq value: 4.568692756483254, max joint entropy: 5.093574258026979
bald curr bs 8, current acq value: 4.8769702677644515, max joint entropy: 5.533326006404283
bald curr bs 9, current acq value: 5.0507129426766255, max joint entropy: 5.8201608968650085
bald curr bs 10, current acq value: 5.2456095263762545, max joint entropy: 6.063712465623093
bald curr bs 11, current acq value: 5.379287114261541, max joint entropy: 6.267493870004331
bald curr bs 12, current acq value: 5.476347575837411, max joint entropy: 6.40744

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


Experiment 1, Total training points: 110, exp acc: [0.8694158075601375, 0.9666175748649976]
bald curr bs 2, current acq value: 1.5900255068526552, max joint entropy: 1.9600848997435694
bald curr bs 3, current acq value: 2.309094303538716, max joint entropy: 2.90365254252187
bald curr bs 4, current acq value: 2.948938077884137, max joint entropy: 3.8138396977023237
bald curr bs 5, current acq value: 3.5337601909898773, max joint entropy: 4.521613510328742
bald curr bs 6, current acq value: 4.0730732561677225, max joint entropy: 5.270985633016565
bald curr bs 7, current acq value: 4.487609811955082, max joint entropy: 6.018116835037619
bald curr bs 8, current acq value: 4.811072612369128, max joint entropy: 6.6107069700800505
bald curr bs 9, current acq value: 5.0298570049189335, max joint entropy: 7.104963333684264
bald curr bs 10, current acq value: 5.17536344299256, max joint entropy: 7.584318040688491
bald curr bs 11, current acq value: 5.327673833270998, max joint entropy: 7.8871104

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


...

Experiment 1, Total training points: 810, exp acc: [0.8694158075601375, 0.9666175748649976, 0.9867452135493373, 0.9911634756995582, 0.9941089837997055, 0.9950908198330879, 0.9950908198330879, 0.9965635738831615, 0.993127147766323]
bald curr bs 2, current acq value: 1.4934620801175862, max joint entropy: 1.9225722197259694
bald curr bs 3, current acq value: 2.1043413492898546, max joint entropy: 2.8452247604650287
bald curr bs 4, current acq value: 2.692822487104109, max joint entropy: 3.6847708532692702
bald curr bs 5, current acq value: 3.206939637213612, max joint entropy: 4.537952715229886
bald curr bs 6, current acq value: 3.5998296115679107, max joint entropy: 5.17029088476127
bald curr bs 7, current acq value: 3.9410558033898466, max joint entropy: 5.843385828928216
bald curr bs 8, current acq value: 4.226238402382097, max joint entropy: 6.354732393773908
bald curr bs 9, current acq value: 4.485328285322958, max joint entropy: 6.889933415264233
bald curr bs 10, current acq value:

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


Experiment 1, Total training points: 910, exp acc: [0.8694158075601375, 0.9666175748649976, 0.9867452135493373, 0.9911634756995582, 0.9941089837997055, 0.9950908198330879, 0.9950908198330879, 0.9965635738831615, 0.993127147766323, 0.9970544918998527]
bald curr bs 2, current acq value: 1.3724956116634315, max joint entropy: 1.8899700245815223
bald curr bs 3, current acq value: 1.92963187774527, max joint entropy: 2.788999273170356
bald curr bs 4, current acq value: 2.429303947827161, max joint entropy: 3.5424280799835333
bald curr bs 5, current acq value: 2.895026694692328, max joint entropy: 4.359530697659197
bald curr bs 6, current acq value: 3.3035254546212283, max joint entropy: 5.035051352600965
bald curr bs 7, current acq value: 3.659844853445102, max joint entropy: 5.816069282849916
bald curr bs 8, current acq value: 3.968969862313555, max joint entropy: 6.428147146920774
bald curr bs 9, current acq value: 4.189763168011346, max joint entropy: 7.034799889068867
bald curr bs 10, c

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


Experiment 1, Total training points: 1010, exp acc: [0.8694158075601375, 0.9666175748649976, 0.9867452135493373, 0.9911634756995582, 0.9941089837997055, 0.9950908198330879, 0.9950908198330879, 0.9965635738831615, 0.993127147766323, 0.9970544918998527, 0.997545409916544]
